# Exporting&nbsp;RasterFrames

In [2]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()

samplePath = 'samples/L8-B8-Robinson-IL.tiff'
rf = spark.read.geotiff(samplePath)

While the goal of RasterFrames is to make it as easy as possible to do your geospatial analysis with a single 
construct, it is helpful to be able to transform it into other representations for various use cases.

## Converting to Array

The cell values within a `Tile` are encoded internally as an array. There may be use cases 
where the additional context provided by the `Tile` construct is no longer needed and one would
prefer to work with the underlying array data.

The @scaladoc[`tileToArray`][tileToArray] column function requires a type parameter to indicate the array element
type you would like used. The following types may be used: `Int`, `Double`, `Byte`, `Short`, `Float`

In [4]:
withArrays = rf.withColumn("tileData", tileToArray[Short]('tile')).drop('tile')
withArrays.show(5, 40)

NameError: name 'tileToArray' is not defined

You can convert the data back to an array, but you have to specify the target tile dimensions. 

In [ ]:
tileBack = withArrays.withColumn("tileAgain", arrayToTile($"tileData", 128, 128))
tileBack.drop("tileData").show(5, 40) 

Note that the created tile will not have a `NoData` value associated with it. Here's how you can do that:

In [ ]:
tileBackAgain = withArrays.withColumn("tileAgain", withNoData(arrayToTile($"tileData", 128, 128), 3))
tileBackAgain.drop("tileData").show(5, 50)

## Writing to Parquet

It is often useful to write Spark results in a form that is easily reloaded for subsequent analysis. 
The [Parquet](https://parquet.apache.org/)columnar storage format, native to Spark, is ideal for this. RasterFrames
work just like any other DataFrame in this scenario as long as @scaladoc[`rfInit`][rfInit] is called to register
the imagery types.


Let's assume we have a RasterFrame we've done some fancy processing on: 

In [ ]:
import geotrellis.raster.equalization._
equalizer = udf((t: Tile) => t.equalize())
equalized = rf.withColumn("equalized", equalizer($"tile")).asRF


equalized.printSchema
equalized.select(aggStats($"tile")).show(false)
equalized.select(aggStats($"equalized")).show(false)

We write it out just like any other DataFrame, including the ability to specify partitioning:

In [ ]:
filePath = "/tmp/equalized.parquet"
equalized.select("*", "spatial_key.*").write.partitionBy("col", "row").mode(SaveMode.Overwrite).parquet(filePath)

Let's confirm partitioning happened as expected:

In [ ]:
import java.io.File
new File(filePath).list.filter(f => !f.contains("_"))

Now we can load the data back in and check it out:

In [ ]:
rf2 = spark.read.parquet(filePath)

rf2.printSchema
equalized.select(aggStats($"tile")).show(false)
equalized.select(aggStats($"equalized")).show(false)

## Exporting a Raster

For the purposes of debugging, the RasterFrame tiles can be reassembled back into a raster for viewing. However, 
keep in mind that this will download all the data to the driver, and reassemble it in-memory. So it's not appropriate 
for very large coverages.

Here's how one might render the image to a georeferenced GeoTIFF file: 

In [ ]:
import geotrellis.raster.io.geotiff.GeoTiff
image = equalized.toRaster($"equalized", 774, 500)
GeoTiff(image).write("target/scala-2.11/tut/rf-raster.tiff")

[*Download GeoTIFF*](rf-raster.tiff)

Here's how one might render a raster frame to a false color PNG file:

In [ ]:
val colors = ColorMap.fromQuantileBreaks(image.tile.histogram, ColorRamps.BlueToOrange)
image.tile.color(colors).renderPng().write("target/scala-2.11/tut/rf-raster.png")

![](rf-raster.png)

## Exporting to a GeoTrellis Layer

For future analysis it is helpful to persist a RasterFrame as a [GeoTrellis layer](http://geotrellis.readthedocs.io/en/latest/guide/tile-backends.html).

First, convert the RasterFrame into a TileLayerRDD. The return type is an Either;
the `left` side is for spatial-only keyed data

In [ ]:
tlRDD = equalized.toTileLayerRDD($"equalized").left.get

Then create a GeoTrellis layer writer:

In [ ]:
import java.nio.file.Files
import spray.json._
import DefaultJsonProtocol._
import geotrellis.spark.io._
p = Files.createTempDirectory("gt-store")
writer: LayerWriter[LayerId] = LayerWriter(p.toUri)

layerId = LayerId("equalized", 0)
writer.write(layerId, tlRDD, index.ZCurveKeyIndexMethod)

Take a look at the metadata in JSON format:

In [ ]:
AttributeStore(p.toUri).readMetadata[JsValue](layerId).prettyPrint

## Converting to `RDD` and `TileLayerRDD`

Since a `RasterFrame` is just a `DataFrame` with extra metadata, the method 
@scaladoc[`DataFrame.rdd`][rdd] is available for simple conversion back to `RDD` space. The type returned 
by `.rdd` is dependent upon how you select it.

In [ ]:
import scala.reflect.runtime.universe._
def showType[T: TypeTag](t: T) = println(implicitly[TypeTag[T]].tpe.toString)

showType(rf.rdd)

showType(rf.select(rf.spatialKeyColumn, $"tile".as[Tile]).rdd) 

showType(rf.select(rf.spatialKeyColumn, $"tile").as[(SpatialKey, Tile)].rdd) 

If your goal convert a single tile column with its spatial key back to a `TileLayerRDD[K]`, then there's an additional
extension method on `RasterFrame` called [`toTileLayerRDD`][toTileLayerRDD], which preserves the tile layer metadata,
enhancing interoperation with GeoTrellis RDD extension methods.

In [ ]:
showType(rf.toTileLayerRDD($"tile".as[Tile]))

In [ ]:
```tut:invisible
spark.stop()
```

[rfInit]: astraea.spark.rasterframes.package#rfInit%28SQLContext%29:Unit
[rdd]: org.apache.spark.sql.Dataset#frdd:org.apache.spark.rdd.RDD[T]
[toTileLayerRDD]: astraea.spark.rasterframes.RasterFrameMethods#toTileLayerRDD%28tileCol:RasterFrameMethods.this.TileColumn%29:Either[geotrellis.spark.TileLayerRDD[geotrellis.spark.SpatialKey],geotrellis.spark.TileLayerRDD[geotrellis.spark.SpaceTimeKey]]
[tileToArray]: astraea.spark.rasterframes.ColumnFunctions#tileToArray